In [2]:
import numpy as np
import h5py
import argparse
import sys
import re
import codecs

from collections import Counter

In [3]:
words2index = {}
with open('data/words.dict') as f:
    for line in f:
        (val, key, num) = line.split()
        words2index[key] = int(val)

In [12]:
index2words = {}
with open('data/words.dict') as f:
    for line in f:
        (val, key, num) = line.split()
        index2words[int(val)] = key

In [13]:
def train_get_Ngram(filepath,words2index,N):
    results = []
    with open(filepath) as f:
        for line in f:
            lsplit = [words2index[x] for x in line.split()]
            l = np.append(np.repeat(words2index['<s>'],N-1),lsplit)

            for i in range(len(lsplit)):
                g = l[i:N-1+i]
                v = lsplit[i]
                results.append((g,v))
            results.append((l[-N+1:],words2index['</s>']))
    return results

In [14]:
def valid_test_Ngram(filepath, words2index, N, test=False):
    results = []
    if test == False:
        with open(filepath) as f:
            i = 1
            for line in f:
                lsplit = line.split()
                if lsplit[0] == 'Q':
                    topredict = np.array([words2index[x] for x in lsplit[1:]])
                if lsplit[0] == 'C':
                    l = np.append(
                        np.repeat(words2index['<s>'], N-1), [words2index[x] for x in lsplit[1:-1]])
                    lastNgram = l[-N+1:]
                    results.append((lastNgram, topredict))
    else:
        with open(filepath) as f:
            i = 1
            for line in f:
                lsplit = line.split()
                if lsplit[0] == 'Q':
                    topredict = np.array([words2index[x] for x in lsplit[1:]])
                if lsplit[0] == 'C':
                    l = np.append(
                        np.repeat(words2index['<s>'], N-1), [words2index[x] for x in lsplit[1:-1]])
                    lastNgram = l[-N+1:]
                    results.append((lastNgram, topredict))
    return results

In [15]:
def tomatrix(results,train = True):
    N = len(results[0][0])+1
    if train:
        tuplelist = []
        for i in range(len(results)):
            tuplelist.append(tuple(list(np.append(results[i][0],results[i][1]))))
        Count = Counter(tuplelist).most_common()
        tooutput = np.empty((len(Count),N+1))

        for i in range(len(Count)):
            tooutput[i,:] = np.append(np.array(Count[i][0]),Count[i][1])

        return tooutput.astype(int)
    
    else:
        tooutput = np.empty((len(results),50+N-1))
        
        for i in range(len(results)):
            tooutput[i,:] = np.hstack((results[i][1],results[i][0]))
        
        return tooutput.astype(int)

In [21]:
index2words[8]

'a'

In [16]:
valid_test_Ngram('data/valid_blanks.txt',words2index,6)

[(array([   8,  328, 2517,    7,    1]),
  array([ 407, 6937, 1655,  224, 1573, 6613, 8345, 8618,  297, 9494, 9085,
         2261, 7107, 5195, 8009, 5554, 6273,  922, 7771, 5130, 2408, 9824,
          660, 8177, 5070, 9016, 5779, 1460, 4601, 1555, 4583, 8021, 5839,
         5433, 2645, 5701, 5699, 8434, 4013, 7236,    5,    6, 7391, 3700,
         8474, 2839, 7576,  844, 6710, 4194])),
 (array([3, 3, 3, 2, 2]),
  array([6409, 1971, 9521, 2053, 4732, 2975, 2611, 5007, 8220,  568, 5776,
         9726, 4543, 7588, 6918, 3202, 8849, 4095, 6823, 7029, 6559,  202,
         2837, 8379, 3429, 7196, 4074, 3433, 5862, 5177,  157, 4181,  706,
         8420, 6646, 6425, 4001, 9541, 9158, 1017, 2451, 3556, 8510, 7129,
         5301, 6969, 2190,  221, 8803, 6594])),
 (array([   3,   77,  423,  197, 3884]),
  array([ 500,  305, 8425, 5919, 4090, 4981, 8611, 8244, 4804, 2009, 9910,
          471,    6, 9904, 1198,  514, 1061, 5049, 6289, 8585, 1697, 4648,
         3676, 7371, 1867, 4857, 3924, 6939, 8

In [82]:
bigrams = train_get_Ngram('data/train.txt',2)

In [5]:
trigrams = train_get_Ngram('data/train.txt',words2index,3)

In [8]:
print np.append(trigrams[1][0],trigrams[1][1])

[    3  9981 10001]


In [250]:
tomatrix(trigrams).shape

(589720, 4)

In [24]:
def validation_kaggle(filepath):
    it = 0
    results = []
    with open(filepath) as f:
        for line in f:
            if it == 0 :
                it+=1
            else:
                lsplit = line.split(',')
                l = [int(x.rstrip()) for x in lsplit[1:]]
                results.append(l)
    return np.array(results)
            
        

In [33]:
valid_test_Ngram('data/valid_blanks.txt',words2index,2)

[(array([1]),
  array([ 407, 6937, 1655,  224, 1573, 6613, 8345, 8618,  297, 9494, 9085,
         2261, 7107, 5195, 8009, 5554, 6273,  922, 7771, 5130, 2408, 9824,
          660, 8177, 5070, 9016, 5779, 1460, 4601, 1555, 4583, 8021, 5839,
         5433, 2645, 5701, 5699, 8434, 4013, 7236,    5,    6, 7391, 3700,
         8474, 2839, 7576,  844, 6710, 4194])),
 (array([2]),
  array([6409, 1971, 9521, 2053, 4732, 2975, 2611, 5007, 8220,  568, 5776,
         9726, 4543, 7588, 6918, 3202, 8849, 4095, 6823, 7029, 6559,  202,
         2837, 8379, 3429, 7196, 4074, 3433, 5862, 5177,  157, 4181,  706,
         8420, 6646, 6425, 4001, 9541, 9158, 1017, 2451, 3556, 8510, 7129,
         5301, 6969, 2190,  221, 8803, 6594])),
 (array([3884]),
  array([ 500,  305, 8425, 5919, 4090, 4981, 8611, 8244, 4804, 2009, 9910,
          471,    6, 9904, 1198,  514, 1061, 5049, 6289, 8585, 1697, 4648,
         3676, 7371, 1867, 4857, 3924, 6939, 8431, 3086, 4288, 3138, 4486,
         3949, 3730, 8607, 7310, 9

### Dealing with 1000 words:

In [4]:
words2index1000 = {}
with open('data/words.1000.dict') as f:
    for line in f:
        (val, key, num) = line.split()
        words2index1000[key] = int(val)

In [6]:
def unk_or_index(word,words2index1000):
    if word in words2index1000.keys():
        return words2index1000[word]
    else:
        return words2index1000['<unk>']

In [5]:
results = []
with open('data/valid.1000.txt') as f:
    for line in f:
        lsplit = [words2index1000[x] for x in line.split()]
        l = np.append(np.repeat(words2index1000['<s>'],N-1),lsplit[:-1])
        

2

In [8]:
[1,2,3,4][:-1]

[1, 2, 3]

In [6]:
np.array([[1,1],[1,1]])

array([[1, 1],
       [1, 1]])

In [33]:
# Case N=1: ie prior on the word distribution from the train text

def get_prior(filepath, words2index):
    counter = Counter()
    with open(filepath) as f:
        lines = f.readlines()
        for line in lines:
            # Adding the end of line prediction
            lsplit = line.split() + ['</s>']
            counter.update(lsplit[1:])
    # Build count matrix: (N_words, 2), col 1: word index, col2: word cout
    count_matrix = np.zeros((len(counter), 2), dtype=int)
    
    for i,t in enumerate(counter.iteritems()):
        k, v = t
        count_matrix[i, 0] = words2index[k]
        count_matrix[i, 1] = v
    return count_matrix
        

In [31]:
count_matrix = get_prior('data/train.txt', words2index)

In [32]:
count_matrix

array([[5053,   12],
       [5884,   10],
       [6722,    9],
       ..., 
       [7826,    8],
       [6278,   11],
       [6721,    1]])

In [34]:
# Saving count_matrix
filename = '1-grams.hdf5'
with h5py.File(filename, "w") as f:
    f['train'] = count_matrix